# Download YouTube Dataset
**Tự động download và giải nén**

In [ ]:
!pip install kagglehub -q

In [ ]:
import kagglehub
import os

print("📥 Downloading...")
csv_path = kagglehub.dataset_download(
    "rsrishav/youtube-trending-video-dataset",
    path="US_youtube_trending_data.csv"
)
json_path = kagglehub.dataset_download(
    "rsrishav/youtube-trending-video-dataset",
    path="US_category_id.json"
)

In [ ]:
import zipfile
import gzip
import shutil

os.makedirs('./data', exist_ok=True)

print("Processing...")

# Detect file type và giải nén
def extract_file(input_path, output_path):
    with open(input_path, 'rb') as f:
        magic = f.read(2)
    
    try:
        # ZIP file (PK)
        if magic == b'PK':
            print("  → Detected: ZIP")
            with zipfile.ZipFile(input_path, 'r') as z:
                names = z.namelist()
                csv_name = [n for n in names if n.endswith('.csv')][0]
                with z.open(csv_name) as zf:
                    with open(output_path, 'wb') as out:
                        shutil.copyfileobj(zf, out)
        # GZIP file
        elif magic == b'\x1f\x8b':
            print("  → Detected: GZIP")
            with gzip.open(input_path, 'rb') as gz:
                with open(output_path, 'wb') as out:
                    shutil.copyfileobj(gz, out)
        # Plain CSV
        else:
            print("  → Detected: Plain file")
            shutil.copy(input_path, output_path)
    except Exception as e:
        print(f"  ⚠️ Error: {e}")
        shutil.copy(input_path, output_path)

extract_file(csv_path, './data/raw_data.csv')
shutil.copy(json_path, './data/category_id.json')


In [ ]:
import pandas as pd

df = pd.read_csv('./data/raw_data.csv')
print(f"Dataset: {len(df):,} rows x {len(df.columns)} cols")
print(f"\nColumns:\n{list(df.columns)}\n")
print("First 3 rows:")
df.tail(3)


In [ ]:
# Sort data theo trending_date (mới nhất → cũ nhất)
df['trending_date'] = pd.to_datetime(df['trending_date'])
df_sorted = df.sort_values('trending_date', ascending=False)

print("Date Range:")
print(f"  Newest: {df_sorted['trending_date'].iloc[0]}")
print(f"  Oldest: {df_sorted['trending_date'].iloc[-1]}")
print(f"\nTop 5 most recent videos:")
df_sorted.head(5)[['trending_date', 'title', 'channelTitle', 'view_count']]